In [1]:
from importlib import reload
import os
from toolz import pipe as p

import matplotlib.pyplot as plt
import numpy as np

from tensorboardX import SummaryWriter

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms

import trainModel
import visualizeModel

In [2]:
%matplotlib inline

In [3]:
device = torch.device("cuda")

# Load Images

In [10]:
reload(trainModel)

def createDataloaders(data_dir, input_size=224): 
    xs = ['train', 'val']

    data_transforms = trainModel.create_data_transforms(input_size, input_size)
    
    image_datasets = {x: p(data_dir, 
                           lambda _:os.path.join(_, x),
                           lambda _: datasets.ImageFolder(_, data_transforms[x])
                          )
                      for x in xs}

    dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4,
                                                 shuffle=True, num_workers = 4)
                   for x in xs}

    dataset_sizes = {x: len(image_datasets[x]) for x in xs}
    
    return image_datasets, dataloaders, dataset_sizes

In [11]:
data_dir = 'scrap_data400/'
image_datasets, dataloaders, dataset_sizes = createDataloaders(data_dir)

In [12]:
class_names = image_datasets['train'].classes

In [14]:
class_names

['gossiping',
 'isolation',
 'laughing',
 'nonbullying',
 'pullinghair',
 'punching',
 'quarrel',
 'slapping',
 'stabbing',
 'strangle']

In [15]:
n_classes = len(class_names)

In [ ]:
dataset_sizes

# Visualize

In [ ]:
reload(visualizeModel)

inputs, classes = next(iter(dataloaders['train']))

out = torchvision.utils.make_grid(inputs)

visualizeModel.imshow(out, mn, sd, 
                      title=[class_names[x] for x in classes])

# Finetune Convnet

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
model

In [ ]:
reload(trainModel)
trainModel.viewParamsToBeUpdated(model)

In [ ]:
reload(trainModel)

model = trainModel.train_model(
    model, criterion, optimizer, exp_lr_scheduler, dataloaders, 
    dataset_sizes,
    device, num_epochs=25)

## Visualize Results

In [ ]:
reload(trainModel)
trainModel.visualize_model(model, dataloaders, device, class_names, 
                           trainModel.defaultMn, 
                           trainModel.defaultSd, k='val')

# Hyper Parameter Search

In [5]:
def makeHyperParamName(heir_array):
    '/'.join(heir_array)

In [ ]:
trainModel.viewParamsToBeUpdated(model)

In [66]:
reload(trainModel)

def trainWithParams(runDir, fine_tune_params_filter = [],  
                    model_type="resnet", 
                    dataloaders = dataloaders, dataset_sizes = dataset_sizes,
                    n_epochs = 25):
    
    model = trainModel.initialize_model(model_type, n_classes,
                                               True)
    model = model.to(device)
    

    p(
        trainModel.findParam(model, name_filter = fine_tune_params_filter),
        lambda named_params: [param_value for (_, param_value) in named_params],
        lambda pvs: trainModel.setParameterRequiresGrad(
            model, params = pvs, requires_grad = True)
    )
        
    writer = p('runs/' + runDir, SummaryWriter)
    
    criterion = nn.CrossEntropyLoss()

    # Observe that all parameters are being optimized
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

    # Decay LR by a factor of 0.1 every 7 epochs
    exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

    model = trainModel.train_model(
        model, criterion, optimizer, exp_lr_scheduler, 
        dataloaders, dataset_sizes, device, 
        num_epochs=n_epochs, writer=writer)
    
    return model

In [40]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [48]:
train_ns = [400, 1000, 1500]
tune_params = [
    ['fc'],
    ['layer4.1.bn2'],
    ['layer4.1', '2'],
    ['layer4.1'],
    ['layer4']
]

In [55]:
[pn for (pn, _) in trainModel.findParam(model, tune_params[0])]

['fc.weight', 'fc.bias']

In [57]:
['_'.join(tp) for tp in tune_params]

['fc', 'layer4.1.bn2', 'layer4.1_2', 'layer4.1', 'layer4']

In [70]:
model_type = 'resnet'
models = dict()
for tp in tune_params:
    for train_n in train_ns:
        runDir = '/'.join(['resnet', '_'.join(tp), str(train_n)])
        print(runDir)
        
        _, dataloaders, dataset_sizes = createDataloaders('scrap_data' + str(train_n))
        
        model = trainWithParams(runDir, fine_tune_params_filter = tp,  
                                model_type=model_type, 
                                dataloaders = dataloaders, dataset_sizes = dataset_sizes,
                               n_epochs=25)
        models[runDir] = model

resnet/fc/400
Epoch 0/24
----------
train Loss: 52.8875 Acc: 0.2400
val Loss: 35.1101 Acc: 0.4075
Epoch 1/24
----------
train Loss: 49.2948 Acc: 0.3775
val Loss: 54.5494 Acc: 0.4525
Epoch 2/24
----------
train Loss: 50.9292 Acc: 0.3875
val Loss: 31.7185 Acc: 0.5175
Epoch 3/24
----------
train Loss: 36.9582 Acc: 0.4850
val Loss: 43.5114 Acc: 0.4825
Epoch 4/24
----------
train Loss: 42.7479 Acc: 0.5075
val Loss: 43.6218 Acc: 0.4875
Epoch 5/24
----------
train Loss: 36.5536 Acc: 0.5125
val Loss: 51.8088 Acc: 0.4600
Epoch 6/24
----------
train Loss: 38.8645 Acc: 0.5700
val Loss: 67.5058 Acc: 0.4575
Epoch 7/24
----------
train Loss: 18.1961 Acc: 0.7075
val Loss: 40.0352 Acc: 0.5525
Epoch 8/24
----------
train Loss: 10.8150 Acc: 0.7350
val Loss: 40.3950 Acc: 0.5575
Epoch 9/24
----------
train Loss: 11.0269 Acc: 0.7725
val Loss: 37.4548 Acc: 0.5525
Epoch 10/24
----------
train Loss: 10.6461 Acc: 0.7500
val Loss: 36.5021 Acc: 0.5500
Epoch 11/24
----------
train Loss: 8.8391 Acc: 0.7800
val Los

/home/john/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train Loss: 65.9296 Acc: 0.3007
val Loss: 49.1001 Acc: 0.4650
Epoch 1/24
----------


/home/john/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train Loss: 52.8995 Acc: 0.4427
val Loss: 39.8604 Acc: 0.5550
Epoch 2/24
----------


...

train Loss: 0.3591 Acc: 0.8780
val Loss: 1.0358 Acc: 0.7225
Epoch 23/24
----------


/home/john/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train Loss: 0.3704 Acc: 0.8813
val Loss: 1.0047 Acc: 0.6875
Epoch 24/24
----------


/home/john/anaconda3/lib/python3.6/site-packages/PIL/TiffImagePlugin.py:756: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))


train Loss: 0.3754 Acc: 0.8880
val Loss: 1.0115 Acc: 0.6950


In [68]:
models

{'resnet/fc/400': ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu

In [69]:
for (k, m) in models.items():
    print(k)
    print(trainModel.viewParamsToBeUpdated(m))
    

resnet/fc/400
['fc.weight', 'fc.bias']
resnet/fc/1000
['fc.weight', 'fc.bias']
resnet/fc/1500
['fc.weight', 'fc.bias']
resnet/layer4.1.bn2/400
['layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1.bn2/1000
['layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1.bn2/1500
['layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1_2/400
['layer4.1.conv2.weight', 'layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1_2/1000
['layer4.1.conv2.weight', 'layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1_2/1500
['layer4.1.conv2.weight', 'layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1/400
['layer4.1.conv1.weight', 'layer4.1.bn1.weight', 'layer4.1.bn1.bias', 'layer4.1.conv2.weight', 'layer4.1.bn2.weight', 'layer4.1.bn2.bias', 'fc.weight', 'fc.bias']
resnet/layer4.1/1000
['layer4.1.conv1.weight', 'layer4.1.bn1.weight